# Building a Neural Network Regression Model in Keras

By Jaime Hernandez

### Introduction

This notebook experiments with the implementation of regression model using the deep learning Keras library.  Performance of the model is evaluated with an increasing the number of training epochs and changing number of hidden layers. For this analysis, concrete strength will be predicted using this dataset: https://cocl.us/concrete_data.

### Load Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


### Read Data and Gain Understanding of Dataset

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
print("Shape:")
print(concrete_data.shape)
concrete_data.head()

Shape:
(1030, 9)


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
#Check for nulls
print(concrete_data.isnull().sum())


Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64


### Prepare Data For Model - Split into Predictors and Target

In [4]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [7]:
n_cols = predictors.shape[1] # number of predictors

### Building Neural Network Model

One hidden layer of 10 nodes and a ReLU activation function will be used.

In [8]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

### Fit Model and Evaluate

A) Fit using 50 Epochs, 50 separete times using different random train-test samples with a 70:30 split.

In [9]:
#Establish parameters
EPochs = 50
samples = 50

In [11]:
#Predict and evaluate

#Starting list that will contain RMSE scores for each sample
eval_list = []

for i in range(samples):
    #Split data into test and train over each iteration
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=.3, random_state=99)

    #Re-build NN
    model = regression_model()
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPochs, verbose=0)
    eval_list.append(model.evaluate(X_test, y_test, verbose=0))

    

In [12]:
#Calculate mean and standard deviation of RMSE list

mean = sum(eval_list) / len(eval_list) 
variance = sum([((x - mean) ** 2) for x in eval_list]) / len(eval_list) 
st_dev = variance ** 0.5


print("The Mean of the RMSE list is : " +str(mean)+ " and the Standard deviation is : " + str(st_dev)) 


The Mean of the RMSE list is : 527.113924249939 and the Standard deviation is : 748.5367294555975


### Experimenting with Normalized Data

B) Normalize data and compare RMSE with previous outputs

In [13]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [15]:
#Predict and evaluate

#Starting list that will contain RMSE scores for each sample
eval_list = []

for i in range(samples):
    #Split data into test and train over each iteration
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=.3, random_state=99)

    #Re-build NN
    model = regression_model()
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPochs, verbose=0)
    eval_list.append(model.evaluate(X_test, y_test, verbose=0))

    

In [16]:
#Calculate mean and standard deviation of RMSE list

mean = sum(eval_list) / len(eval_list) 
variance = sum([((x - mean) ** 2) for x in eval_list]) / len(eval_list) 
st_dev = variance ** 0.5


print("The Mean of the RMSE list is : " +str(mean)+ " and the Standard deviation is : " + str(st_dev)) 



The Mean of the RMSE list is : 350.3140299558176 and the Standard deviation is : 104.17171565362709


When looking at the means, we can see that by normalizing the data, we were able to decrease or RMSE form about 527 to  350!


### Experimenting with More Epochs

C) Increase the number of Epochs to 100

In [17]:
#Establish parameters
EPochs = 100
samples = 50

In [18]:
#Predict and evaluate

#Starting list that will contain RMSE scores for each sample
eval_list = []

for i in range(samples):
    #Split data into test and train over each iteration
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=.3, random_state=99)

    #Re-build NN
    model = regression_model()
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPochs, verbose=0)
    eval_list.append(model.evaluate(X_test, y_test, verbose=0))

    

In [19]:
#Calculate mean and standard deviation of RMSE list

mean = sum(eval_list) / len(eval_list) 
variance = sum([((x - mean) ** 2) for x in eval_list]) / len(eval_list) 
st_dev = variance ** 0.5


print("The Mean of the RMSE list is : " +str(mean)+ " and the Standard deviation is : " + str(st_dev)) 


The Mean of the RMSE list is : 168.9056067250076 and the Standard deviation is : 10.048759660100451


When looking at the means, we can see that by increasing the epochs from 50 to 100, we were able to decrease or RMSE form about 350 to 168!


### Experiment with More Hidden Layers

D) Based on version B) (only 50 epochs), increase the number of hidden layers to 3, each with 10 nodes, using ReLU activation functions.

In [20]:
# define regression model
def regression_model_2():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))

    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [21]:
#Establish parameters
EPochs = 50 #reseting to 50 epochs
samples = 50

In [22]:
#Predict and evaluate

#Starting list that will contain RMSE scores for each sample
eval_list = []

for i in range(samples):
    #Split data into test and train over each iteration
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=.3, random_state=99)

    #Re-build NN
    model = regression_model_2()
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPochs, verbose=0)
    eval_list.append(model.evaluate(X_test, y_test, verbose=0))

    

In [23]:
#Calculate mean and standard deviation of RMSE list

mean = sum(eval_list) / len(eval_list) 
variance = sum([((x - mean) ** 2) for x in eval_list]) / len(eval_list) 
st_dev = variance ** 0.5


print("The Mean of the RMSE list is : " +str(mean)+ " and the Standard deviation is : " + str(st_dev)) 



The Mean of the RMSE list is : 137.4183537146806 and the Standard deviation is : 13.854536418998409


When looking at the means, we can see that when we keep the same basline of B), 50 Epochs, but we increae the number of hidden layers to 3 with 10 nodes each, we were able to decrease or RMSE form about 352 to 137!


### Conclusion

After experimenting with multiple parameters and types of neural networks, it was determined that normalizing the data is a simple way to increase the performance of our models before even training them. It was also determined that 2 other ways to increase model performance are increasing the number of  epochs and the number of hidden layers. For this problem, increasing the number of hidden layers (D) to 3 yielded the best RMSE; 137.